In [ ]:
from plots.plot_utils import BASELINE_NAMES
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# elevator_lossmpaic1.0_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_8.pkl

missing_files = []
base_path = "/home/scratch/yixiz/results/seeds_10/"
dataset = "facebook"
hp_dir = 'nl-8_hs-256'
import os
for seed in seeds:
    for baseline in BASELINE_NAMES:
        path = f"{base_path}/{dataset}/{hp_dir}/{dataset}_loss{baseline}_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_{seed}_models.pkl"
        if not os.path.exists(path):
            missing_files.append(path)

print(f"Number of missing files: {len(missing_files)}")
for file in missing_files:
    print(file)

In [ ]:
# renaming example:
# diamonds_lossmpaic_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_8_alpha0.4_models.pkl
# to: diamonds_lossmpaic0.4_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_8_models.pkl

import os
from pathlib import Path
base_path = "/home/scratch/yixiz/results/seeds_10/"
base_path = Path(base_path)
for p in base_path.rglob("*_models.pkl"):
    if "mpaic" not in p.stem:
        continue
    stem = p.stem
    parts = stem.split("_")
    loss_part = parts[1]
    alpha_part = parts[-1]
    if "alpha" not in alpha_part:
        continue
    alpha_value = alpha_part.replace("alpha", "")
    new_loss_part = f"{loss_part}{alpha_value}"
    parts.remove(alpha_part)
    parts[1] = new_loss_part
    new_stem = "_".join(parts)
    new_path = p.with_name(new_stem + p.suffix)
    # os.rename(p, new_path)
    print(f"Renamed {p} to {new_path}")


In [ ]:
from pathlib import Path
import pickle
facebook_base = Path("/home/scratch/yixiz/results/seeds_10/facebook/nl-8_hs-256/")
for p in facebook_base.glob("*.pkl"):
    if "models" in p.stem:
        continue
    with open(p, "rb") as f:
        data = pickle.load(f)
    for va_ece, va_sharp_score, te_ece, te_sharp_score in zip(data['va_ece_controlled'], data['va_sharp_score_controlled'], data['te_ece_controlled'], data['te_sharp_score_controlled']):
        if te_sharp_score > 10:
            print(f"File: {p}, va_ece: {va_ece}, va_sharp_score: {va_sharp_score}, te_ece: {te_ece}, te_sharp_score: {te_sharp_score}")

for p in facebook_base.glob("*.pkl"):
    if "models" in p.stem:
        continue
    with open(p, "rb") as f:
        data = pickle.load(f)
    for recal_va_ece, recal_va_sharp_score, recal_te_ece, recal_te_sharp_score in zip(data['recal_va_ece_controlled'], data['recal_va_sharp_score_controlled'], data['recal_te_ece_controlled'], data['recal_te_sharp_score_controlled']):
        if recal_te_sharp_score > 10:
            print(f"File: {p}, recal_va_ece: {recal_va_ece}, recal_va_sharp_score: {recal_va_sharp_score}, recal_te_ece: {recal_te_ece}, recal_te_sharp_score: {recal_te_sharp_score}")

In [ ]:
from pathlib import Path
import pickle
base_path = Path("/home/scratch/yixiz/results/seeds_10/")
for p in base_path.rglob("*.pkl"):
    if "models" in p.stem:
        continue
    with open(p, "rb") as f:
        data = pickle.load(f)
    try:
        data['te_ece'], data['va_ece'], data['recal_te_ece'], data['recal_va_ece'],
        data['te_sharp_score'], data['va_sharp_score'], data['recal_te_sharp_score'], data['recal_va_sharp_score']
    except KeyError as e:
        print(f"File: {p} is missing keys.")
        # p.unlink()

In [ ]:
import pickle
import __main__
from utils.q_model_ens import QModelEns, EnhancedMLP
import torch.nn as nn
class VanillaModel(nn.Module):
    def __init__(self, nfeatures):
        super().__init__()
        self.net = None
    def forward(self, x):
        return self.net(x)
__main__.VanillaModel = VanillaModel
__main__.EnhancedMLP = EnhancedMLP
__main__.QModelEns = QModelEns # Also add any other custom classes
data = pickle.load(open('/home/scratch/yixiz/results/seeds_10/protein/nl-8_hs-256/protein_losscalipso_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_0.pkl', 'rb'))

In [ ]:
import pickle
with open('/home/scratch/yixiz/eval/concrete_lossbatch_qr_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl8_hs256_0_crossing_quantile.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
for te_ece, te_sharp_score, avg_swaps in zip(data['te_ece'], data['te_sharp'], data['avg_swaps']):
    print(f"te_ece: {te_ece}, te_sharp_score: {te_sharp_score}, avg_swaps: {avg_swaps}")
import matplotlib.pyplot as plt
plt.scatter(data['te_ece'], data['avg_swaps'])
plt.xlabel('Test ECE')
plt.ylabel('Average Swaps')
plt.show()
plt.close()
plt.scatter(data['te_sharp'], data['avg_swaps'])
plt.xlabel('Test Sharp Score')
plt.ylabel('Average Swaps')

In [ ]:
from pathlib import Path
from plots.plot_utils import DATASETS
import pickle
import numpy as np

datasets_min_te_ece_stats = {dataset: (None, None, []) for dataset in DATASETS}
RESULTS_BASE_PATH = Path("/home/scratch/yixiz/results/seeds_10/")

for dataset in DATASETS:
    dataset_path = RESULTS_BASE_PATH / dataset
    min_te_eces = []
    for p in dataset_path.rglob("*.pkl"):
        if 'models' in p.stem or 'batch_qr' not in p.stem:
            continue
        with open(p, "rb") as f:
            data = pickle.load(f)
        min_te_ece = min(data['va_ece_controlled'])
        min_te_eces.append(min_te_ece)
    if min_te_eces:
        avg_min_te_ece = sum(min_te_eces) / len(min_te_eces)
        std_min_te_ece = (sum((x - avg_min_te_ece) ** 2 for x in min_te_eces) / len(min_te_eces)) ** 0.5

        datasets_min_te_ece_stats[dataset] = (avg_min_te_ece, std_min_te_ece, min_te_eces)
for dataset, (avg_min_te_ece, std_min_te_ece, min_te_eces) in datasets_min_te_ece_stats.items():
    print(f"Dataset: {dataset}, Avg Min TE ECE: {avg_min_te_ece}, Std Min TE ECE: {std_min_te_ece}")

Dataset: boston, Avg Min TE ECE: 0.04392874110490084, Std Min TE ECE: 0.015470524071534644
Dataset: concrete, Avg Min TE ECE: 0.025512761436402798, Std Min TE ECE: 0.004182514030066842
Dataset: energy, Avg Min TE ECE: 0.023648569360375404, Std Min TE ECE: 0.0036886160818685927
Dataset: kin8nm, Avg Min TE ECE: 0.022025679424405097, Std Min TE ECE: 0.0022669351739593078
Dataset: naval, Avg Min TE ECE: 0.01370246270671487, Std Min TE ECE: 0.003205847252023007
Dataset: power, Avg Min TE ECE: 0.008581835450604558, Std Min TE ECE: 0.0015816200188059507
Dataset: protein, Avg Min TE ECE: 0.00936722601763904, Std Min TE ECE: 0.002219625052403654
Dataset: wine, Avg Min TE ECE: 0.027066778019070627, Std Min TE ECE: 0.004973460911070206
Dataset: yacht, Avg Min TE ECE: 0.02446464542299509, Std Min TE ECE: 0.0034135954593017613
Dataset: diamonds, Avg Min TE ECE: 0.007946220086887478, Std Min TE ECE: 0.0010442865220906962
Dataset: elevator, Avg Min TE ECE: 0.012152439169585704, Std Min TE ECE: 0.0009

In [ ]:
import pickle
from pathlib import Path
# concrete_lossbatch_qr_optadam_ens1_bootFalse_resTrue_lnFalse_bnFalse_dr0.0_lr0.005_bs64_nl8_hs256_0_models.pkl
base_path = Path("/home/scratch/yixiz/results/optim/")
for p in base_path.rglob("*_models.pkl"):
    with open(p, "rb") as f:
        data = pickle.load(f)
    print(f"File: {p}, len: {len(data['models'])}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data['va_ece_list'][:1000])

In [ ]:
from pathlib import Path
import shutil
base_path = Path("/home/scratch/yixiz/results/optim/")
optimizers = ['adam', 'sgd', 'adamw', 'rmsprop']
for opt in optimizers:
    for dataset_dir in base_path.iterdir():
        if not dataset_dir.is_dir():
            continue
        (dataset_dir / f'opt-{opt}').mkdir(exist_ok=True)

for dataset_dir in base_path.iterdir():
    if not dataset_dir.is_dir():
        continue
    for pkl in (dataset_dir / 'nl-8_hs-256').glob("*.pkl"):
        if not any(opt in pkl.stem for opt in optimizers):
            continue
        opt = next(opt for opt in optimizers if opt in pkl.stem)
        shutil.copy(pkl, dataset_dir / f'opt-{opt}' / pkl.name)




In [ ]:
from pathlib import Path
import shutil
base_path = Path("/home/scratch/yixiz/results/optim/")
lrs = [str(lr) for lr in [5e-4, 1e-3, 5e-3, 1e-2]]
for lr in lrs:
    for dataset_dir in base_path.iterdir():
        if not dataset_dir.is_dir():
            continue
        (dataset_dir / f'lr-{lr}').mkdir(exist_ok=True)

for dataset_dir in base_path.iterdir():
    if not dataset_dir.is_dir():
        continue
    for pkl in (dataset_dir / 'nl-8_hs-256').glob("*.pkl"):
        if not any(lr in pkl.stem for lr in lrs):
            continue
        lr = next(lr for lr in lrs if lr in pkl.stem)
        shutil.copy(pkl, dataset_dir / f'lr-{lr}' / pkl.name)